<a href="https://colab.research.google.com/github/Rob174/Cours/blob/master/tp%20rendre%2015%20janvier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programmation linéaire - Travaux pratiques A rendre pour le 15 janvier à

tom.pontoleav@laas.fr
laurent.chasserat@laas.fr

**<font color="red">Groupe :</font>** 3 MIC C

**<font color="red">Étudiants :</font>** Robin Moine / ...

In [0]:
#Cellule permettant sur Google Colab le rendu de Latex dans les sorties de cellules de code
from IPython.display import HTML
def typeset():
  """MathJax initialization for the current cell.
  
  This installs and configures MathJax for the current output.
  """
  display(HTML('''
      <script src="https://www.gstatic.com/external_hosted/mathjax/latest/MathJax.js?config=TeX-AMS_HTML-full,Safe&delayStartupUntil=configured"></script>
      <script>
        (() => {
          const mathjax = window.MathJax;
          mathjax.Hub.Config({
          'tex2jax': {
            'inlineMath': [['$', '$'], ['\\(', '\\)']],
            'displayMath': [['$$', '$$'], ['\\[', '\\]']],
            'processEscapes': true,
            'processEnvironments': true,
            'skipTags': ['script', 'noscript', 'style', 'textarea', 'code'],
            'displayAlign': 'center',
          },
          'HTML-CSS': {
            'styles': {'.MathJax_Display': {'margin': 0}},
            'linebreaks': {'automatic': true},
            // Disable to prevent OTF font loading, which aren't part of our
            // distribution.
            'imageFont': null,
          },
          'messageStyle': 'none'
        });
        mathjax.Hub.Configured();
      })();
      </script>
      '''))


# 1. Introduction

Le but de ces travaux pratiques est d'implémenter l'algorithme du simplexe en python en utilisant la méthode des dictionnaires vue en cours.

## 1.1. Rappels de cours

Soit le programme linéaire $(\mathcal{P})$ suivant:

$$
\mathcal{P}=\left(\begin{array}{rrrlrlrl}
\max & z = & 4 x_1 & + &  3x_2 & \\
s.c. & & x_1 & &  &  \leq & 8 \\
& & x_1 &  +  & 2x_2 & \leq & 15 \\
& & 2x_1&  + & x_2 & \leq & 18 \\
& & \rlap{x_j \geq 0 \quad j= 1,2}
\end{array}\right.
$$

**Exercice :** Mettre le problème sous forme standard (contraintes d'égalité).

$$
\mathcal{P}=\left(\begin{array}{rrrlrlrl}
\max & z = & 4 x_1 & + & 3x_2 & +&0x_3&+&0x_4&+&0x_5\\
s.c. & & x_1 & + &  & +&x_3 && &&&= & 8 \\
 & & x_1 & + & 2x_2 &  &&+& x_4&&&= & 15 \\
 & & 2x_1 & + & x_2 &  && &&+&x_5&= & 18 \\
& & \rlap{x_j \geq 0 \quad j= 1,2,3,4,5}
\end{array}\right.
$$

**Exercice :** Donner une base évidente pour le problème ci-dessus.


**<font color="red">Réponse étudiant(e)s:</font>**

(0,0,8,15,18)

**Exercice :** Écrire le dictionnaire correspondant à la base de la question précédente.

**<font color="red">Réponse étudiant(e)s:</font>**

$$
\begin{array}{r||r|r|r|r|}
 & b & x_1 & x_2 \\\hline
 x_3 & 8 & -1 & 0\\
 x_4 & 15 & -1 & -2\\
 x_5 & 18 & -2 & -1 \\\hline\hline
 z & 0 & 4 & 3\\\hline
\end{array}
$$


## 1.2 &mdash; Implémentation des dictionnaires en python

### 1.2.1. La structure python `simplex_dictionary`

Nous allons utilisés la structure `simplex_dictionary` afin de représenter un dictionnaire du simplex en python en python.
Cette structure est fournie par le package `simplex` qui peut être installé en exécutant la cellule ci-dessous.

<div class="alert alert-block alert-info">
    
Il est indispensable de vérifier que le package `simplex` a été installé correctement et que la classe `simplex_dictionary`
est disponible avant de passer à la suite.
Si la cellule s'exécute sans erreurs, c'est que la classe a été importée correctement.
</div>

<div class="alert alert-block alert-warning">
    <b>Attention !</b> Il existe une structure <code>dict</code> très courante en python appelée « dictionary », qui est
    utilisée dans la classe <code>simplex_dictionary</code> pour stocker certaines valeurs. À ne pas confondre avec
    notre classe <code>simplex_dictionary</code>.
</div>

In [1]:
# Install the small simplex package:
!pip install --user git+https://gitea.typename.fr/mikael.capelle/simplex.git
    
# Import the simplex_dictionary class:
from simplex import simplex_dictionary

  Cloning https://gitea.typename.fr/mikael.capelle/simplex.git to /tmp/pip-req-build-28fbzk4g
  Running command git clone -q https://gitea.typename.fr/mikael.capelle/simplex.git /tmp/pip-req-build-28fbzk4g
  Created wheel for simplex: filename=simplex-0.0.1-cp36-none-any.whl size=7077 sha256=24296f7fd283b619afbfd18c368f79e1a9fa49a5d3a490675a48a23093f92fd9
  Stored in directory: /tmp/pip-ephem-wheel-cache-k1x401_4/wheels/60/1c/89/223b1fa2de37798acbffd20dbb3c9bb846ff1f7a54b42cc05c
Successfully built simplex


### 1.2.2. Création d'un dictionnaire en python

Soit le programme linéaire suivant sous forme normale:

$$
\begin{array}{rrrlrlrlrlrlrl}
\max & z = & 3 x_1 & + & 2 x_2 &   &     &   &    \\
s.c.     & & 2 x_1 & + &   x_2 & + & x_3 & = & 18  \\
         & & 2 x_1 & + & 3 x_2 & + & x_4 & = & 42 \\
         & & 3 x_1 & + &   x_2 & + & x_5 & = & 24 \\
& & \rlap{x_j \geq 0 \quad j= 1, 2, 3, 4, 5}
\end{array}
$$

Ce programme possède une base évidente $\mathcal{B}_0$ composée des variables d'écarts $\mathcal{B}_0 = (x_4, x_5, x_6)$. On peut donc écrire le dictionaire suivant:

$$
\begin{array}{lrlrlrlrlrlr}
 z  & = &  0 & + & 3x_1 & + & 2x_2 \\
x_3 & = & 18 & - & 2x_1 & - &  x_2 \\
x_4 & = & 42 & - & 2x_1 & - & 3x_2 \\
x_5 & = & 24 & - & 3x_1 & - &  x_2
\end{array}
$$

Ou sous forme de tableau:

$$
\begin{array}{r||r|r|r|r|}
 & b & x_1 & x_2 \\\hline
 x_3 & 18 & -2 & -1\\
 x_4 & 42 & -2 & -3\\
 x_5 & 24 & -3 & -1 \\\hline\hline
 z & 0 & 3 & 2\\\hline
\end{array}
$$

Le code python ci-dessous permet de créer un dictionnaire python (`simplex_dictionary`) représentant le dictionnaire ci-dessus.

<div class="alert alert-block alert-info">
    
Il est possible d'afficher un object `simplex_dictionary` sous forme mathématique via la méthode
`.display()`. 
Pour obtenir un affiche cohérent, il est recommandé de nommer les variables utilisées de la forme
`a_n` où `a` est une lettre et `n` un nombre, par exemple `x_1` ou `y_3`.
    
</div>

In [117]:
# The list of variables:
x1, x2, x3, x4, x5 = ('x_{}'.format(i + 1) for i in range(5))
variables = [x1, x2, x3, x4, x5]
print('Variables:', variables)

# The simplex dictionary with B = (x3, x4, x5):
sdict = simplex_dictionary(B=[x3, x4, x5], N=[x1, x2])

# Set the values of the basic variables:
sdict.b = {x3: 18, x4: 42, x5: 24}

# Coefficients of the non-basic variables in the dictionary (we represent
# the positive coefficients):
sdict.a = {
    x3: {x1: 2, x2: 1},
    x4: {x1: 2, x2: 3},
    x5: {x1: 3, x2: 1}
}

# Current value of the objective:
sdict.z = 0

# Coefficients of the non-basic variables in the objective function:
sdict.c[x1] = 3
sdict.c[x2] = 2

# Display the dictionary:
typeset()
sdict.display(name='S_0')

Variables: ['x_1', 'x_2', 'x_3', 'x_4', 'x_5']


<IPython.core.display.Math object>

## 2. Implémentation de l'algorithme du simplexe en python

### 2.1. Pré-calcul

On considère maintenant le dictionnaire de l'exemple précédent :

$$
\begin{array}{r||r|r|r|r|}
 & b & x_1 & x_2 \\\hline
 x_3 & 18 & -2 & -1\\
 x_4 & 42 & -2 & -3\\
 x_5 & 24 & -3 & -1 \\\hline\hline
 z & 0 & 3 & 2\\\hline
\end{array}
$$

**Exercice :** Trouver la variable entrante $x_k$ et sortante $x_s$, et effectuer une étape de l'algorithme
du simplexe sur le dictionnaire ci-dessus.

$x_1$ rentre en base et $x_5$ sort de base

**Exercice :** Construire le nouveau dictionnaire, `ndict`, obtenu après l'étape de pivotage.

<div class="alert alert-block alert-info">

Afin d'éviter les erreurs d'arrondi inhérentes aux calculs en nombres flottants, il est impossible d'utiliser 
la classe `simplex_dictionary` avec des valeurs flottantes (par exemple `0.33`).
Pour représenter de manière exacte des nombres rationnels en python, tel que $\frac{1}{3}$, vous pouvez
utiliser la class `Fraction` (du package `fractions`). Par exemple:

```python
from fractions import Fraction
Fraction(1, 3)
```

</div>

In [119]:
from fractions import Fraction

# On construit un dictionnaire avec la nouvelle base:
ndict = simplex_dictionary(B=[x3, x4, x1], N=[x2,x5])

# On définit les valeurs des variables b_i et des coefficients a_ij:
ndict.b = {x3: 2, x4: 26, x1: 8}
ndict.a = {
    x3: {x2: Fraction(1,3), x5: Fraction(-2,3)},
    x4: {x2: Fraction(7,3), x5: Fraction(-2,3)},
    x1: {x2: Fraction(1,3), x5: Fraction(1,3)}
}

# On définit le nouvel objectif et les nouveaux coefficients:
ndict.z = 24
ndict.c = {
    x2:1,
    x5:-1
}

# On affiche le dictionnaire:
typeset()
ndict.display()

<IPython.core.display.Math object>

### 2.2. Recherche des variables entrantes et sortantes en python

Nous allons maintenant implémenter les fonctions python suivantes:

- `find_entering_variable` &mdash; Permet de trouver la variable entrante pour un dictionnaire donné.
- `find_leaving_variable` &mdash; Permet de trouver la variable sortante pour un dictionnaire et une variable
    entrante donnés.
    
Dans les deux cas, nous utiliserons la [règle de Bland](https://en.wikipedia.org/wiki/Bland%27s_rule) en 
cas de choix (règle du plus petit indice).

Les fonctions retourneront la valeur python `None` si aucune variable n'est possible, par exemple, si
tous les coefficients sont négatifs dans l'expression de l'objectif.

**Exercice :** Écrire la fonction `find_entering_variable` ci-dessous et vérifier qu'elle retourne $x_k$ sur le dictionnaire initial `sdict`.

<div class="alert alert-block alert-info">

Il existe une fonction `min` en python qui fonctionne sur tous les types, et en particulier sur les 
chaînes de caractères.

</div>

In [4]:
def find_entering_variable(sdict):
    """ Retrieve the index of the next entering variable from the given
    simplex dictionary using Bland's rule.
    
    Parameters:
      - sdict The simplex dictionary to use.
      
    Returns: The next entering variable, or None if there
    is none.
    """    
    x = min(sdict.c)
    if sdict.c[x] <= 0:
        return None
    return x
xk = find_entering_variable(sdict)
print('Entering variable:', xk)

Entering variable: x_1


**Exercice :** Écrire la fonction `find_leaving_variable` ci-dessous et vérifier qu'elle retourne $x_s$ sur le dictionnaire initial `sdict` et $x_k$.

<div class="alert alert-block alert-info">

Les valeurs des variables dans `b` et des coefficients dans `a` sont stockées sous forme
d'object `fractions.Fraction` dans un `simplex_dictionary`, ce qui permet de ne pas perdre en
précision.
En particulier, diviser une valeur par une autre produit un autre object `fractions.Fraction`, et
il est donc possible de comparer de manière exacte (`==`) deux nombre rationnel:

```python
a = Fraction(3, 8)
b = Fraction(6, 14)

assert a / b == Fraction(7, 8)
```

</div>

In [5]:
sdict.a

{'x_3': {'x_1': Fraction(2, 1), 'x_2': Fraction(1, 1)},
 'x_4': {'x_1': Fraction(2, 1), 'x_2': Fraction(3, 1)},
 'x_5': {'x_1': Fraction(3, 1), 'x_2': Fraction(1, 1)}}

In [6]:
def find_leaving_variable(sdict, xk):
    """ Retrieve the index of the next leaving variable from the given
    simplex dictionary and entering variable using Bland's rule.
    
    Parameters:
      - sdict  The simplex dictionary to use.
      - xk     The next entering variable.
      
    Returns: The next leaving variable, or None if there is none (i.e., 
    the problem is unbounded).
    """
    x_sortant = None
    valmin = 10**7
    for ligne_x in sdict.b:
        b = sdict.b[ligne_x]
        dicta = sdict.a[ligne_x]
        if b/dicta[xk] < valmin:
            x_sortant = ligne_x
            valmin = b/dicta[xk]
    return x_sortant

xs = find_leaving_variable(sdict, xk)
print('Leaving variable:', xs)

Leaving variable: x_5


### 2.3. Pivot du dictionnaire à partir des variables entrantes et sortantes en python

**Exercice :** Écrire la fonction `pivot_dictionary` ci-dessous.

In [7]:
sdict.B

['x_3', 'x_4', 'x_5']

In [0]:
def pivot_dictionary(sdict, xk, xs):
    """ Pivot the given dictionary on the given row / column and creates a new one. 
    
    Parameters:
      - sdict  The simplex dictionary to use.
      - xk     The entering variable.
      - xs     The leaving variable.
      
    Returns: A new simplex dictionary after the pivot operation. 
    """
    # On construit un dictionnaire avec la nouvelle base:
    B = []
    print("Sort de base ",xs)
    print("Rentre en base ",xk)
    for x in sdict.b:
        if x != xs:
            B.append(x)
    B.append(xk)
    N = []
    for x in sdict.a[xs]:
        if x != xk:
            N.append(x)
    N.append(xs)
    print("Varibles en bases : ",','.join(B)," et hors base : ",','.join(N))
    ndict = simplex_dictionary(B=B, N=N)
    #On calcul la valeur de la variable entrante xk = ....
    dictxk =  {k: v/sdict.a[xs][xk] for k, v in sdict.a[xs].items()}#Copie du dictionnaire de la variable sortante
    del dictxk[xk]#Suppression du coefficient de la variable entrant en base
    dictxk[xs] = 1/sdict.a[xs][xk]
    ndict.a[xk] = dictxk
    print(ndict.a)
    #On ajoute le coefficient de xk
    dicob = {}
    dicob[xk] = sdict.b[xs]/sdict.a[xs][xk]
    #Avec cela on écrit la valeur de xk dans les autres lignes
    for ligne_x in sdict.a:
        if ligne_x != xs:#Si on a une ligne différente de celle de la variable entrante
            #On remplace la valeur de la variable sortante par son expresion trouvée ci_dessus
            #Ce qui peut changer les coefficients des autres variables
            for col_x in sdict.a[ligne_x]:
                dic = {}
                if col_x != xk:#On passe l'index de la variable entrante qui ne sera plus dans ndict.a
                    dic[col_x] = sdict.a[ligne_x][col_x] + dictxk[col_x]*sdict.a[ligne_x][xk]
            dic[xs] = dictxk[xs]*sdict.a[ligne_x][xk]#On ajoute le coefficient de la variable sortante
            ndict.a[ligne_x] = dic
            #Et la valeur de la constante
            dicob[ligne_x] = sdict.b[ligne_x] + sdict.b[xs]*sdict.a[ligne_x][xk]
    print("a ",ndict.a)
    ndict.b = dicob
    print("b ",ndict.b)
    # On change également l'objectif          
    dictc = {}
    for x in sdict.c:
        if x in ndict.a[xk]:
            dictc[x] = sdict.c[x]+ndict.a[xk][x]*sdict.c[xk]
            
    dictc[xs] = ndict.a[xk][xs]*sdict.c[xk]
    ndict.c = dictc
    print("c ",ndict.c)
    ndict.z = sdict.z + ndict.b[xk]*sdict.c[xk]
    return ndict

 **Exercie:** Vérifier que l'appel `pivot_dictionary(sdict, xk, xs)` retourne bien le dictionnaire trouver à la section **2.1**.
 NB : on prend alors la même règle de choix de variable entrante

In [125]:
xk = find_entering_variable(sdict)
xs = find_leaving_variable(sdict, xk)
ndict = pivot_dictionary(sdict, xk, xs)
typeset()
ndict.display()

Sort de base  x_5
Rentre en base  x_1
Varibles en bases :  x_3,x_4,x_1  et hors base :  x_2,x_5
{'x_1': {'x_2': Fraction(1, 3), 'x_5': Fraction(1, 3)}}
a  {'x_1': {'x_2': Fraction(1, 3), 'x_5': Fraction(1, 3)}, 'x_3': {'x_2': Fraction(5, 3), 'x_5': Fraction(2, 3)}, 'x_4': {'x_2': Fraction(11, 3), 'x_5': Fraction(2, 3)}}
b  {'x_1': Fraction(8, 1), 'x_3': Fraction(66, 1), 'x_4': Fraction(90, 1)}
c  {'x_2': Fraction(3, 1), 'x_5': Fraction(1, 1)}


<IPython.core.display.Math object>

### 2.4. Algorithme du simplex (une phase) en python

**Exercice :** Écrire la méthode `simplex_single_phase` ci-dessous qui, à partir d'un dictionnaire `sdict` (que l'on considéra valide, i.e., $b_i \geq 0, \forall i\in\mathcal{B}$), retourne le dictionnaire final de l'algorithme.

<div class="alert alert-block alert-info">

N'hésitez pas à afficher les dictionnaires intermédaires via la
méthode `.display()` pour visualiser l'évolution de l'algorithme.

</div>

In [0]:
def simplex_single_phase(sdict, log=True):
    """ Apply the simplex algorithm on the given dictionary.
    
    Parameters:
      - sdict  The initial dictionary to start the algorithm. Must be valid.
      
    Return: A tuple (z, dict) containing the value of the objective (or None is the problem
    is not bounded), and the final dictionary (or any dictionary if the problem is unbounded).
    """
    pass

**Exercice :** Vérifier que votre méthode `simplex_single_phase` trouve bien la solution optimale pour le 
problème défini dans la section **2.1**.

In [0]:
z, d = simplex_single_phase(sdict)

## 3. Implémentation de l'algorithme du simplexe en 2 phases en python

### 3.1. Pré-calcul

On considère maintenant le programme linéaire suivant :

$$
\mathcal{P}=\left(\begin{array}{rrrlrlrl}
\max & z = & - x_1 & - &  x_2 & \\
s.c. & & -3x_1 & - & 4x_2 &  \leq & -12 \\
& & 2x_1 &  +  & x_2 & \leq & 4 \\
& & \rlap{x_j \geq 0 \quad j= 1,2}
\end{array}\right.
$$

Dont la représentation graphique est donnée ci-dessous:
    
![simplex_2_phases.PNG](attachment:simplex_2_phases.PNG)
        
Le problème sous forme normale s'écrit :

$$
\mathcal{P}=\left(\begin{array}{rrrlrlrlrl}
\max & z = & - x_1 & - &  x_2 & & & \\
s.c. & & -3x_1 & - & 4x_2 & + & x_3 & = & -12 \\
& & 2x_1 &  +  & x_2 & + & x_4 & = & 4 \\
& & \rlap{x_j \geq 0 \quad j= 1,2,3,4}
\end{array}\right.
$$

La base $\mathcal{B} = \{x_3, x_4\}$ n'est pas donc pas une base réalisable, mais on peut tout de même 
écrire un dictionnaire pour cette base:

$$
\begin{array}{r||r|r|r|r|}
 & b & x_1 & x_2 \\\hline
 x_3 & -12 & -3 & -4\\
 x_4 & 4 & 2 & 1\\
 z & 0 & -1 & -1\\\hline
\end{array}
$$

**Exercice :** Construire en python le dictionnaire ci-dessus.

### 3.2. Vérification d'un dictionnaire

On souhaite maintenant écrire une fonction `is_valid_dictionary` qui vérifie qu'une dictionnaire est valide, i.e., que toutes les variables de bases sont positives ou nulles.

**Exercice :** Écrire la fonction `is_valid_dictionary` et vérifier qu'elle retourne `False` pour le dictionnaire défini ci-dessus.

In [0]:
def is_valid_dictionary(sdict):
    """ Check if the given dictionary is valid, i.e., if all the basic variables are positive or 0.
    
    Parameters:
      - sdict  The dictionary to check.
      
    Return: True if the dictionary is valid, false otherwise.
    """
    pass

is_valid_dictionary(sdict)

### 3.3. Recherche d'un dictionnaire initial valide

Puisque le dictionnaire que l'on a construire à partir de la base triviale composée des variables d'écart
n'est pas valide, il faut en trouver un valide, ou s'assurer qu'il n'en existe pas, auquel cas le problème
original serait infaisable.

Afin de trouver un dictionnaire valide, nous allons utiliser un programme linéaire auxiliaire en décomposant
les variables d'écarts négatives en deux variables, e.g., $x_3' = x_3 - y$.

**Exercice :** Écrire le programme linéaire auxiliaire correspond au problème sous forme normale ci-dessus, 
et le dictionnaire correspondant.

**<font color="red">Réponse étudiant(e)s:</font>**

**Exercire :** Écrire la fonction `make_auxiliary_dictionary` en python qui, à partir d'un dictionnaire non-valide,
construit le dictionnaire du programme linéaire auxiliaire associé. Vérifier que l'appel `make_auxiliary_dictionary(sdict)` retourne le dictionnaire écrit ci-dessus.


<div class="alert alert-block alert-info">
    On nommera les variables auxiliaires $y_1$, $y_2$, $\ldots{}$.
</div>

In [0]:
def make_auxiliary_dictionary(sdict):
    """ Create the initial dictionary of the auxiliary program from the given
    dictionary.
    
    Parameters:
      - sdict  The dictionary of the original linear program.
      
    Return: The initial dictionary of the auxiliary linear program.
    """
    pass

In [0]:
ndict = make_auxiliary_dictionary(sdict)
ndict.display()

**Exercice :** En utilisant la fonction `simplex_single_phase` définie dans la section précédente, 
résoudre le problème auxiliaire.

### 3.4. Phase 1 de l'algorithme du simplexe


**Question :** Comment détecte-t-on que le problème original n'a pas de solution ?

**Exercice :** Écrire la fonction `simplex_initial_phase` qui, étant donné un dictionnaire (valide ou non), 
retourne soit un dictionnaire valide pour le problème, soit `None` si le problème est infaisable.

In [0]:
def simplex_initial_phase(sdict, log=False):
    """ Create a valid dictionary corresponding to the same problem as the given dictionary, 
    if possible.
    
    Parameters:
      - sdict  The initial dictionary to start from.
      
    Return: A valid dictionary corresponding to the same program as sdict, or None if the linear
    program is invalid.
    """
    pass

**Exercice:** Utiliser la méthode `simplex_initial_phase` pour obtenir un dictionnaire valide pour le problème défini dans la section **3.1**.

# 4. Algorithme du simplexe en python

**Exercice :** En utilisant les méthodes définies ci-dessus, écrire la fonction `simplex_algorithm` qui, à partir d'un dictionnaire (valide ou non), retourne un tuple `(z, x, d)` où `z` contient la valeur de l'objectif (maximisation), `x` un mapping (python's `dict`) entre les variables et leurs valeurs et `d` est le dictionnaire final. 

**Note :** On lèvera les exceptions `InfeasibleProgram` ou  `UnboundedProgram` si le problème est infaisable ou non-
borné.

<div class="alert alert-block alert-info">
    
Pour lever une instruction en python, on utilise le mot-clé `raise`:

```python
raise InfeasibleProgram()
```

</div>

In [0]:
class InfeasibleProgram(Exception): pass
class UnboundedProgram(Exception): pass

def simplex_algorithm(sdict, log=False):
    """ Solve the linear program corresponding to the given simplex dictionary.
    
    Parameters:
      - sdict  The initial simplex dictionary. May be invalid.
      
    Return: A tuple (z, x, d) where z is the optimal value of the objective function, and
    x is a mapping (python dictionary) from variables to their value.
    
    Raise:
      - InfeasibleProgram if the program corresponding to the dictionary is infeasible.
      - UnboundedProgram if the program corresponding to the dictionary is not bounded.
    """
    pass

In [0]:
z, x, d = simplex_algorithm(sdict)
d.display()